#ETL File to Generate the Lab Features


This file generates the lab features based on concept queries detailed in MIMIC-III folder. https://github.com/MIT-LCP/mimic-code/tree/master/concepts
The table leverages several pivot tables from this code respository:
lab_pivot, pivot_bp, and pivot_soft

Having access to the underling files would required an access request for this project: my-project-1553961115123
Please contact paulhaolee@gmail.com for access request


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .appName('Jupyter BigQuery Storage')\
  .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
  .getOrCreate()

In [ ]:
import pyspark
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
from pyspark.sql.window import Window
from datetime import datetime


In [ ]:
#Hardcode schema to speed up data read
#Code Reference:https://github.com/MIT-LCP/mimic-code/blob/master/buildmimic/aws-athena/mimictoparquet_glue_job.py
schema_icustays = StructType([
    StructField("row_id", IntegerType()),
    StructField("subject_id", IntegerType()),
    StructField("hadm_id", IntegerType()),
    StructField("icustay_id", IntegerType()),
    StructField("dbsource", StringType()),
    StructField("first_careunit", StringType()),
    StructField("last_careunit", StringType()),
    StructField("first_wardid", ShortType()),
    StructField("last_wardid", ShortType()),
    StructField("intime", TimestampType()),
    StructField("outtime", TimestampType()),
    StructField("los", DoubleType())
])

schema_patients = StructType([
    StructField("row_id", IntegerType()),
    StructField("subject_id", IntegerType()),
    StructField("gender", StringType()),
    StructField("dob", TimestampType()),
    StructField("dod", TimestampType()),
    StructField("dod_hosp", TimestampType()),
    StructField("dod_ssn", TimestampType()),
    StructField("expire_flag", IntegerType())
])

schema_services = StructType([
    StructField("row_id", IntegerType()),
    StructField("subject_id", IntegerType()),
    StructField("hadm_id", IntegerType()),
    StructField("transfertime", TimestampType()),
    StructField("prev_service", StringType()),
    StructField("curr_service", StringType())
])

schema_chartevents = StructType([
    StructField("row_id", IntegerType()),
    StructField("subject_id", IntegerType()),
    StructField("hadm_id", IntegerType()),
    StructField("icustay_id", IntegerType()),
    StructField("itemid", IntegerType()),
    StructField("charttime", TimestampType()),
    StructField("storetime", TimestampType()),
    StructField("cgid", IntegerType()),
    StructField("value", StringType()),
    StructField("valuenum", DoubleType()),
    StructField("valueuom", StringType()),
    StructField("warning", IntegerType()),
    StructField("error", IntegerType()),
    StructField("resultstatus", StringType()),
    StructField("stopped", StringType())
])


schema_ditems = StructType([
    StructField("row_id", IntegerType()),
    StructField("itemid", IntegerType()),
    StructField("label", StringType()),
    StructField("abbreviation", StringType()),
    StructField("dbsource", StringType()),
    StructField("linksto", StringType()),
    StructField("category", StringType()),
    StructField("unitname", StringType()),
    StructField("param_type", StringType()),
    StructField("conceptid", IntegerType())
])

schema_admissions = StructType([
    StructField("row_id", IntegerType()),
    StructField("subject_id", IntegerType()),
    StructField("hadm_id", IntegerType()),
    StructField("admittime", TimestampType()),
    StructField("dischtime", TimestampType()),
    StructField("deathtime", TimestampType()),
    StructField("admission_type", StringType()),
    StructField("admission_location", StringType()),
    StructField("discharge_location", StringType()),
    StructField("insurance", StringType()),
    StructField("language", StringType()),
    StructField("religion", StringType()),
    StructField("marital_status", StringType()),
    StructField("ethnicity", StringType()),
    StructField("edregtime", TimestampType()),
    StructField("edouttime", TimestampType()),
    StructField("diagnosis", StringType()),
    StructField("hospital_expire_flag", ShortType()),
    StructField("has_chartevents_data", ShortType())
])


schema_lab_evnts = StructType([
    StructField("row_id", IntegerType()),
    StructField("subject_id", IntegerType()),
    StructField("hadm_id", IntegerType()),
    StructField("itemid", IntegerType()),
    StructField("charttime", TimestampType()),
    StructField("value", StringType()),
    StructField("valuenum", DoubleType()),
    StructField("valueuom", StringType()),
    StructField("flag", StringType())
])

In [ ]:
df_icustays = spark.read.csv('gs://hlee-data/mimiciii_physionet/ICUSTAYS.csv', sep = ',', schema = schema_icustays, header = True)
df_admissions = spark.read.csv('gs://hlee-data/mimiciii_physionet/ADMISSIONS.csv', sep = ',', schema = schema_admissions, header = True)
df_lab_events = spark.read.csv('gs://hlee-data/mimiciii_physionet/LABEVENTS.csv', sep = ',', schema = schema_lab_evnts, header = True)


In [ ]:
#Create Temporary Tables for query
df_icustays.registerTempTable('icustays')
df_admissions.registerTempTable('admissions')
df_lab_events.registerTempTable('lab_evnts')

In [ ]:
schemaSepPat = StructType([
    StructField("ic", IntegerType()),
    StructField("hd", IntegerType()),
    StructField("sepsis", IntegerType())
])


In [ ]:
sepsis = pd.read_csv("gs://hlee-data/misc/sepsis3-df.csv")

In [ ]:
cols = ['icustay_id', 'hadm_id', 'sepsis-3']
sepPat = sepsis[cols]
sepPat.columns = ['ic', 'hd', 'sepsis-3']

In [ ]:
df_pat = spark.createDataFrame(sepPat,schema=schemaSepPat)


###ETL Step for generating features based on the lap_pivot table from MIMICIII

---



In [ ]:
table = "my-project-1553961115123.aa.lab_pivot"
df = spark.read \
  .format("bigquery") \
  .option("table", table) \
  .load()
df.printSchema()

In [ ]:
#join the lab_pivot table with the Sepsis data for contorl and case patients identified
labevnts = df.join(df_pat, (df.icustay_id == df_pat.ic) &
    (df.hadm_id == df_pat.hd), how = "inner")

#establish the min time of the lab_pivot file per icu_stay_id
windowSpec  = Window.partitionBy("icustay_id")
labevnts =labevnts.withColumn("start_date",min(col("charttime")).over(windowSpec))
labevnts = labevnts.withColumn("stay_mins", (unix_timestamp(col("charttime"), "yyyy-MM-dd'T'HH:mm") - unix_timestamp(col("start_date"), "yyyy-MM-dd'T'HH:mm"))/60)

#output the data to a BigQuery table under my project
labevnts.write \
  .format("bigquery") \
  .option("temporaryGcsBucket","hlee-data/misc/") \
  .save("aa.lab_sepsis_patients")



###ETL Step for generating features based on the pivot_soft table from MIMICIII


In [ ]:
table = "my-project-1553961115123.aa.pivot_soft"
df = spark.read \
  .format("bigquery") \
  .option("table", table) \
  .load()
df.printSchema()

root
 |-- icustay_id: long (nullable = true)
 |-- hr: long (nullable = true)
 |-- starttime: string (nullable = true)
 |-- endtime: string (nullable = true)
 |-- pao2fio2ratio_novent: double (nullable = true)
 |-- pao2fio2ratio_vent: double (nullable = true)
 |-- rate_epinephrine: double (nullable = true)
 |-- rate_norepinephrine: double (nullable = true)
 |-- rate_dopamine: double (nullable = true)
 |-- rate_dobutamine: double (nullable = true)
 |-- meanbp_min: double (nullable = true)
 |-- GCS_min: double (nullable = true)
 |-- urineoutput: double (nullable = true)
 |-- bilirubin_max: double (nullable = true)
 |-- creatinine_max: double (nullable = true)
 |-- platelet_min: double (nullable = true)
 |-- respiration: long (nullable = true)
 |-- coagulation: long (nullable = true)
 |-- liver: long (nullable = true)
 |-- cardiovascular: long (nullable = true)
 |-- cns: long (nullable = true)
 |-- renal: long (nullable = true)
 |-- respiration_24hours: long (nullable = true)
 |-- coagulat

In [ ]:
labevnts = df.join(df_pat, (df.icustay_id == df_pat.ic) , how = "inner")

In [ ]:
table = "my-project-1553961115123.aa.lab_sepsis_patients"
lab_sepsis = spark.read \
  .format("bigquery") \
  .option("table", table) \
  .load()
# lab_sepsis.printSchema()
columns_to_drop = ['ic', 'hd', 'sepsis', 'start_date', 'stay_mins']
lab_sepsis= lab_sepsis.drop(*columns_to_drop)

In [ ]:
from pyspark.sql.functions import date_trunc
## round lab_spsis time to hours
lab_sepsis = lab_sepsis.withColumn('charttime', date_trunc('hour', lab_sepsis.charttime))

In [ ]:
## round sofa time to hours
labevnts.cache()
lab_time = labevnts.withColumn('starttime',   from_unixtime(unix_timestamp('starttime', "yyyy-MM-dd'T'HH:00")))
lab_time = lab_time.withColumn('endtime',   from_unixtime(unix_timestamp('endtime', "yyyy-MM-dd'T'HH:00")))

lab_time = lab_time.withColumn('starttime', date_trunc('hour', lab_time.starttime))

In [ ]:
lab_sepsis.cache()

DataFrame[icustay_id: bigint, hadm_id: bigint, subject_id: bigint, charttime: timestamp, ANIONGAP: double, ALBUMIN: double, BANDS: double, BICARBONATE: double, BILIRUBIN: double, CREATININE: double, CHLORIDE: double, GLUCOSE: double, HEMATOCRIT: double, HEMOGLOBIN: double, LACTATE: double, PLATELET: double, POTASSIUM: double, PTT: double, INR: double, PT: double, SODIUM: double, BUN: double, WBC: double]

In [ ]:
#join labsepsis with the time of lab dataframe created above
lab_sepsis = lab_sepsis.withColumnRenamed("icustay_id", "lab_icu")
sofa_pivot = lab_time.join(lab_sepsis, (lab_time.starttime == lab_sepsis.charttime) & (lab_time.icustay_id == lab_sepsis.lab_icu), how = 'left')
sofa_pivot.cache()

In [ ]:
#select only the columns that are needed for the predictive model
sofa_pivot = sofa_pivot.select(col("icustay_id"), col("hr"), col("starttime"), col("endtime"), col("BUN"), \
                               col("CREATININE"), col("liver"), col("liver_24hours"), col("renal"), col("renal_24hours"), col("WBC"), col("PLATELET"))

##Collect the data from the pivot_bg table which include  other lab measures

In [ ]:
table = "my-project-1553961115123.aa.pivot_bg"
lab_bg = spark.read \
  .format("bigquery") \
  .option("table", table) \
  .load()
lab_bg.printSchema()

root
 |-- icustay_id: long (nullable = true)
 |-- hadm_id: long (nullable = true)
 |-- charttime: string (nullable = true)
 |-- SPECIMEN: string (nullable = true)
 |-- AADO2: double (nullable = true)
 |-- BASEEXCESS: double (nullable = true)
 |-- BICARBONATE: double (nullable = true)
 |-- TOTALCO2: double (nullable = true)
 |-- CARBOXYHEMOGLOBIN: double (nullable = true)
 |-- CHLORIDE: double (nullable = true)
 |-- CALCIUM: double (nullable = true)
 |-- GLUCOSE: double (nullable = true)
 |-- HEMATOCRIT: double (nullable = true)
 |-- HEMOGLOBIN: double (nullable = true)
 |-- INTUBATED: double (nullable = true)
 |-- LACTATE: double (nullable = true)
 |-- METHEMOGLOBIN: double (nullable = true)
 |-- O2FLOW: double (nullable = true)
 |-- FIO2: double (nullable = true)
 |-- SO2: double (nullable = true)
 |-- PCO2: double (nullable = true)
 |-- PEEP: double (nullable = true)
 |-- PH: double (nullable = true)
 |-- PO2: double (nullable = true)
 |-- POTASSIUM: double (nullable = true)
 |-- REQ

In [ ]:
lab_bg = lab_bg.select(col('icustay_id'), col('charttime'), col('PH'), col('PO2'))
lab_bg = lab_bg.withColumn('charttime',   from_unixtime(unix_timestamp('charttime', "yyyy-MM-dd'T'HH:mm")))
lab_bg.cache()


In [ ]:
#roud the time to hours
lab_bg = lab_bg.withColumn('charttime', date_trunc('hour', lab_bg.charttime))
lab_bg = lab_bg.withColumnRenamed("icustay_id", "bg_icu")


In [ ]:
#join the lab_bg file with the main lab file
sofa_pivot_bg = sofa_pivot.join(lab_bg, (sofa_pivot.starttime == lab_bg.charttime) & (sofa_pivot.icustay_id == lab_bg.bg_icu), how = 'left')
sofa_pivot_bg.cache()

In [ ]:
# sofa_pivot_bg = sofa_pivot_bg.fillna(0, subset=['BUN', 'CREATININE', 'liver', 'WBC', 'PLATELET', 'PH', 'PO2'])


###Fill missing values

In [ ]:
##Values are forward fill for null values inbetween 2 measurement points
##null values that happened before measurements are filled with the average value
window = Window.partitionBy('icustay_id')\
       .orderBy('hr')\
       .rowsBetween(-1000000, 0)

final = sofa_pivot_bg.\
           withColumn('BUN', last('BUN',ignorenulls = True).over(window))

In [ ]:
colsfill = ['CREATININE', 'liver', 'renal', 'WBC', 'PLATELET', 'PH', 'PO2']

In [ ]:
for col in colsfill:
    final = final.\
               withColumn(col, last(col,ignorenulls = True).over(window))    

In [ ]:
columns_to_drop = ['bg_icu', 'charttime']
final= final.drop(*columns_to_drop)

In [ ]:
#this is the function to fill values that happened before valid measure with the mean
def fill_with_mean(df, exclude=set()): 
    stats = df.agg(*(
        avg(c).alias(c) for c in df.columns if c not in exclude
    ))
    return df.na.fill(stats.first().asDict())

fillfinal = fill_with_mean(final, ["icustay_id", "hr", "starttime", 'endtime'])

In [ ]:
fillfinal.filter(fillfinal.icustay_id == 200087).show()

+----------+---+-------------------+-------------------+------------------+-----------------+-----+-------------+-----+-------------+------------------+------------------+-----------------+------------------+
|icustay_id| hr|          starttime|            endtime|               BUN|       CREATININE|liver|liver_24hours|renal|renal_24hours|               WBC|          PLATELET|               PH|               PO2|
+----------+---+-------------------+-------------------+------------------+-----------------+-----+-------------+-----+-------------+------------------+------------------+-----------------+------------------+
|    200087|  0|2196-08-30 11:00:00|2196-08-30 12:00:00|28.152882455995954|1.352157116774805|    0|            0|    0|            0|11.780703283501285|240.35345743955955|7.405810115133871|116.74611033436886|
|    200087|  1|2196-08-30 12:00:00|2196-08-30 13:00:00|28.152882455995954|1.352157116774805|    0|            0|    0|            0|11.780703283501285|240.35345743

In [ ]:
#output the data to my bigquery project
fillfinal.write \
  .format("bigquery") \
  .option("temporaryGcsBucket","hlee-data/misc/") \
  .save("aa.lab_final")